# Worldwide Gross Models

## Import libraries

In [167]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm

## Getting the data

In [168]:
df_movies = pd.read_csv('../data/processed/pixar_movies_complete.csv')
df_movies

,Year Released,Movie,Length,RT Score,IMDB Score,Metacritic Score,Opening Weekend,Worldwide Gross,Domestic Gross,International Gross,Domestic %,International %,Production Budget,Oscars Nominated,Oscars Won
0,1995,Toy Story,81,100,8.3,92,29.14,362.00,191.80,170.20,52.98,47.02,30,3.0,0.0
1,1998,A Bug's Life,96,92,7.2,77,33.26,363.40,162.80,200.60,44.80,55.20,45,1.0,0.0
2,1999,Toy Story 2,92,100,7.9,88,57.39,485.00,245.90,239.20,50.70,49.32,90,1.0,0.0
3,2001,"Monsters, Inc.",90,96,8.1,78,62.58,528.80,255.90,272.90,48.39,51.61,115,3.0,1.0
4,2003,Finding Nemo,104,99,8.2,90,70.25,895.60,339.70,555.90,37.93,62.07,94,4.0,1.0
5,2004,The Incredibles,115,97,8.0,90,70.47,631.40,261.40,370.00,41.40,58.60,92,4.0,2.0
6,2006,Cars,116,74,7.2,73,60.12,462.00,244.10,217.90,52.84,47.16,70,2.0,0.0
7,2007,Ratatouille,111,96,8.0,96,47.00,623.70,206.40,417.30,33.09,66.91,150,5.0,1.0
8,2008,WALL-E,97,96,8.4,94,63.10,521.30,223.80,297.50,42.93,57.07,180,6.0,1.0
9,2009,Up,96,98,8.3,88,68.11,731.30,293.00,438.30,40.07,59.93,175,5.0,2.0


## Droping columns

In [169]:
df_movies.drop(['Domestic Gross', 'International Gross', 'Domestic %', 'International %', 'Oscars Nominated', 'Oscars Won', 'Movie'], axis=1, inplace=True)
df_movies

,Year Released,Length,RT Score,IMDB Score,Metacritic Score,Opening Weekend,Worldwide Gross,Production Budget
0,1995,81,100,8.3,92,29.14,362.00,30
1,1998,96,92,7.2,77,33.26,363.40,45
2,1999,92,100,7.9,88,57.39,485.00,90
3,2001,90,96,8.1,78,62.58,528.80,115
4,2003,104,99,8.2,90,70.25,895.60,94
5,2004,115,97,8.0,90,70.47,631.40,92
6,2006,116,74,7.2,73,60.12,462.00,70
7,2007,111,96,8.0,96,47.00,623.70,150
8,2008,97,96,8.4,94,63.10,521.30,180
9,2009,96,98,8.3,88,68.11,731.30,175


## Model with Raw DataFrame

In [170]:
X = df_movies[['IMDB Score', 'Opening Weekend']].values
y = df_movies['Worldwide Gross'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [177]:
sc_x = StandardScaler().fit(X)
sc_y = StandardScaler().fit(y)

X_train = sc_x.transform(X_train)
X_test = sc_x.transform(X_test)
y_train = sc_y.transform(y_train)
y_test = sc_y.transform(y_test)

In [178]:
X_train

array([[-11.43012404,  -1.5462132 ],
       [-13.80530411,  -1.58579645],
       [ -9.76749799,  -1.58025526],
       [-12.61771408,  -1.57837091],
       [-10.95508803,  -1.58361766],
       [-12.14267806,  -1.56887257],
       [-12.14267806,  -1.59437615],
       [-12.61771408,  -1.59782688],
       [-13.09275009,  -1.6174124 ],
       [-10.24253401,  -1.5760449 ],
       [-10.005016  ,  -1.57730507],
       [ -9.76749799,  -1.55245512],
       [-15.23041216,  -1.5876278 ],
       [-10.48005201,  -1.58056147],
       [-10.71757002,  -1.58973593],
       [-11.66764205,  -1.50983335],
       [-10.71757002,  -1.6174124 ],
       [-10.71757002,  -1.57591535],
       [-14.75537614,  -1.55322653],
       [-12.14267806,  -1.6174124 ]])

### Sklearn

In [172]:
sk_model = LinearRegression(fit_intercept=False)
sk_model.fit(X_train, y_train)
y_pred_sk = sk_model.predict(X_test)

In [173]:
mse = mean_squared_error(y_test, y_pred_sk)
r2 = r2_score(y_test, y_pred_sk)

print('r2: ', r2)
print('mse: ', mse)

r2:  0.8645450413935158
mse:  0.0925130753195842


In [174]:
from scipy import stats

params = np.append(sk_model.intercept_, sk_model.coef_)
predictions = sk_model.predict(X_train)

newX = np.append(np.ones((len(X_train), 1)), X_train, axis=1)
MSE = (sum((y_train - predictions) ** 2)) / (len(newX) - len(newX[0]))

var_b = MSE * (np.linalg.inv(np.dot(newX.T, newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params / sd_b

p_values = [2 * (1 - stats.t.cdf(np.abs(i), (len(newX) - 1))) for i in ts_b]

# Print the p-values for the coefficients
print(p_values)

[1.0, 0.0002550220158463734, 3.63197472097454e-10]
